# [CMI-SleepState-Detection](https://www.kaggle.com/competitions/child-mind-institute-detect-sleep-states/overview)
## Child Mind Institute - Detect Sleep States
### Detect sleep onset and wake from wrist-worn accelerometer data
_______________________________________________________________________ 
# Author Details:
- Name: Najeeb Haider Zaidi
- Email: zaidi.nh@gmail.com
- Profiles: [Github](https://github.com/snajeebz)  [LinkedIn](https://www.linkedin.com/in/najeebz) [Kaggle](https://www.kaggle.com/najeebz)
- License: Private, Unlicensed, All the files in this repository under any branch are Prohibited to be used commercially or for personally, communally or privately unless permitted by author in writing.
- Copyrights 2023-2024 (c) are reserved only by the author: Najeeb Haider Zaidi
________________________________________________________________________
# Attributions:
The Dataset has been provided by Child Mind Institute. in [Kaggle Competition](https://www.kaggle.com/competitions/child-mind-institute-detect-sleep-states/overview) which the author is participating in and authorized to use the dataset solely for the competition purposes.
________________________________________________________________________

In [3]:
!pip install pandarallel


  Preparing metadata (setup.py) ... done
  Created wheel for pandarallel: filename=pandarallel-1.6.5-py3-none-any.whl size=16672 sha256=c0b39f6de50e4436293b0fd5a1a806da3e3d464d6723c9815404989476052030
  Stored in directory: /root/.cache/pip/wheels/50/4f/1e/34e057bb868842209f1623f195b74fd7eda229308a7352d47f
Successfully built pandarallel


In [4]:
import numpy as np # linear algebra
import pandas as pd# data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
from pandarallel import pandarallel
import plotly.express as px
import matplotlib.pyplot as plt
from datetime import datetime as dts
pd.set_option('display.max_row', 500)
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_colwidth', None)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
from tqdm import tqdm
tqdm.pandas()
pandarallel.initialize(progress_bar=True)

/kaggle/input/child-mind-institute-detect-sleep-states/train_series.parquet
/kaggle/input/child-mind-institute-detect-sleep-states/sample_submission.csv
/kaggle/input/child-mind-institute-detect-sleep-states/train_events.csv
/kaggle/input/child-mind-institute-detect-sleep-states/test_series.parquet
/kaggle/input/later-data/Full_merged.pkl
/kaggle/input/later-data/full_clustered .pkl
/kaggle/input/later-data/cluster_model.joblib
/kaggle/input/60min-window-before-and-after-the-event/ts_win_60_diff-30.pkl
INFO: Pandarallel will run on 2 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [ ]:
events=pd.read_csv('/kaggle/input/child-mind-institute-detect-sleep-states/train_events.csv')

In [ ]:
df_full=pd.read_pickle('/kaggle/input/later-data/full_clustered .pkl')

# Analyzing the relationshiip between sleep and cluster

In [ ]:
df_sleep=df.loc[df['sleep']==1]
df_onset=df.loc[df['sleep']==0]

In [ ]:
df_sleep['cluster'].mode()

In [ ]:
df_onset['cluster'].mode()

In [ ]:
#df['timestamp']=df['timestamp'].parallel_apply(lambda x: pd.to_datetime(x,utc=True))

In [ ]:
df.info()

In [ ]:
df.to_pickel('cluster_timestamped.pkl')

In [ ]:
events

In [ ]:
events['step'].isnull().sum()

In [ ]:
event_df=events.dropna()

In [ ]:
event_df

In [ ]:
df['window']=np.nan

In [ ]:
ind=df.index[df['event'].isna()==False]
print(ind)

In [ ]:
# 30 min window on change event, with single timestep:5sec. 
c=0
for i in tqdm(ind):
    a=i-360
    b=i+360
    df['window'].loc[a:i]=int(c)
    c=c+1
    df['window'].loc[i:b]=int(c)
    c=c+1
print('Windows Created: ', c)    



In [ ]:
df=df[df['window'].isna()==False]

In [ ]:
df['timestamp']=df['timestamp'].parallel_apply(lambda x: pd.to_datetime(x,utc=True))

# Adding rolling Deviation

In [16]:
def prepare(df):
    df['sd_anglez_1']=pd.to_numeric(df['anglez'].rolling(window=12).std())
    df['sd_enmo_1']=pd.to_numeric(df['enmo'].rolling(window=12).std())
    df['sd_anglez_1'].fillna('0.0',inplace=True)
    df['sd_enmo_1'].fillna('0.0',inplace=True)
    df['sleep'] = df['sleep'].replace({0:False, 0.2:True})
    return(df)

## Analyzing the windows

In [6]:
def visualization(df_n):
    for i in range(10):
        title="Sleep: "+ str(df_n['sleep'][df_n['window']==i].mean())
        fig=px.line(y=df_n['sleep'][df_n['window']==i],x=df_n['timestamp'][df_n['window']==i])
        fig.add_scatter(y=df_n['cluster'][df_n['window']==i],x=df_n['timestamp'][df_n['window']==i], name='cluster')
        fig.add_scatter(y=df_n['enmo'][df_n['window']==i],x=df_n['timestamp'][df_n['window']==i], name='sleep')
        fig.update_layout(title=title)
        fig.show()
visualization(df)

# Evaluate Prediction

In [9]:
def evaluate(y_test,ypred):
    from sklearn.metrics import precision_score
    from sklearn.metrics import recall_score
    from sklearn.metrics import f1_score
    from sklearn.metrics import accuracy_score
    from sklearn.metrics import confusion_matrix
    print("Accuracy: ",accuracy_score(y_test,y_pred)) 
    print("Precision Score : ", precision_score(y_test,y_pred)) #precision measures the proportion of true positive predictions among all positive instances. how many of survived predicted actually survived, doesn't verifies 0's 70 survived as preicted whereas actually 92 survived so 70/92 will be the precision.  if we predicted 70 survived, so presion will tell how many of those 70 predicted survived matches the actual row by row data. It checkes all positives and verifies if the answer is true for each row?
    print("Recall Score: ", recall_score(y_test,y_pred, average='macro')) #Recall measures the proportion of true positive predictions among all actual positive instalnces. If we predicted 100 survived correctly whereas actually 100 survived out of which 67 predicted correctly so recall will be 0.67
    print("F1 Score: ",f1_score(y_test,y_pred)) #mean of recall and precision
    cm=confusion_matrix(y_test, y_pred)
    figure= px.imshow(cm,text_auto=True, width=1200, height=1200)
    figure.show()

In [ ]:
df.groupby(df['sleep']).describe().transpose()

In [17]:
#df.to_pickle('ts_win_60_diff.pkl')
path="/kaggle/input/60min-window-before-and-after-the-event/ts_win_60_diff-30.pkl"
df=prepare(pd.read_pickle(path))


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6910640 entries, 4632 to 127935924
Data columns (total 11 columns):
 #   Column       Dtype              
---  ------       -----              
 0   series_id    object             
 1   step         uint32             
 2   timestamp    datetime64[ns, UTC]
 3   anglez       float32            
 4   enmo         float32            
 5   event        object             
 6   sleep        bool               
 7   cluster      float64            
 8   window       float64            
 9   sd_anglez_1  object             
 10  sd_enmo_1    object             
dtypes: bool(1), datetime64[ns, UTC](1), float32(2), float64(2), object(4), uint32(1)
memory usage: 507.5+ MB


In [31]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
X=df[['sd_anglez_1','sd_enmo_1','anglez','enmo','cluster']]
y=df[['sleep']]
#scaler=preprocessing.StandardScaler().fit(X)
#X_scaled = scaler.transform(X)
X_train, X_test, y_train, y_test =train_test_split(X,y,test_size=0.2, random_state=42)

In [30]:
X_scaled

array([[-0.70038658, -0.46119272,  0.72521887, -0.41692932, -1.19235055],
       [-0.70038658, -0.46119272,  0.87908916, -0.3855034 , -1.19235055],
       [-0.70038658, -0.46119272,  1.01495396, -0.40029207, -1.19235055],
       ...,
       [ 0.85273091,  1.55061615,  0.40367029,  3.03252747, -0.39840566],
       [ 1.72978757,  1.54759635,  1.02171893,  1.05454316, -1.19235055],
       [ 1.76148718,  1.56253419, -0.08986663,  1.52593193, -0.39840566]])

## Import Model

In [ ]:
from joblib import dump, load
model=load('rf_model.joblib')
y_pred=model.predict(X_test)
evaluate(y_test,y_pred)

# Random Forest Classifier

In [32]:
from sklearn.ensemble import RandomForestClassifier
rf =RandomForestClassifier(n_jobs=-1,verbose=1) 
print ('Training the model')
rf.fit(X_train,y_train)
print ('Saving the model')
from joblib import dump, load
dump(rf, 'rf_model.joblib')
y_pred=rf.predict(X_test)
evaluate(y_test,y_pred)


Training the model


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed: 11.5min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed: 26.4min finished


Saving the model


[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:   23.8s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:   53.6s finished


Accuracy:  0.8956442529201347
Precision Score :  0.9084408027976463
Recall Score:  0.8956282796483588
F1 Score:  0.8938628966444948


# SVM Classifier

In [ ]:
from sklearn.svm import SVC
svm = SVC(verbose=1) # n_neighbors is the number of neighbors to consider when predicting a class label for
svm.fit(X_test,y_test)
y_pred=svm.predict(X_test)
evaluate(y_test,y_pred)

[LibSVM]......................

# Naive Bayes Classifier

In [ ]:
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB(n_jobs=-1,verbose=1) 
nb.fit(X_train,y_train)
#dump(nb, 'nb_model.joblib')
y_pred=nb.predict(X_test)
evaluate(y_test,y_pred)

# KNN Classifier

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=10,n_jobs=-1,verbose=1) # n_neighbors is the number of neighbors to consider when predicting a class label for
knn.fit(X_train,y_train)
#dump(knn, 'knn_model.joblib')
y_pred=knn.predict(X_test)
evaluate(y_test,y_pred)

In [ ]:
pred['predicted']=y_pred